In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import random
import os
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [8]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

#### The Data

In [9]:
#train_df = pd.read_csv('../../final/data/train_set_noisy_4_missing_3.csv')
train_df = pd.read_csv('../../final/data/train_set_noisy_6_missing_3.csv')
train_df = train_df.fillna(-1)
train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,9.401651,-1.000000,-1.000000,0.000000,116.415615,103.966653,-1.000000,2.712885,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.0
1,7.029873,102.717562,2.079564,-1.000000,-1.000000,79.373697,212.817785,2.657004,1,0.675576,104.688780,50.452242,-1.000000,-1.000000,49.832026,21.089620,-1.000000,3.0
2,15.900836,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1,0.999932,90.916733,60.845387,77.678965,1.513323,139.638756,47.702509,-1.000000,0.0
3,12.710083,437.896009,-1.000000,-1.000000,392.155888,103.338649,91.089902,3.689834,1,1.431816,31.576192,124.321572,79.407308,24.911028,84.817826,-1.000000,-1.000000,7.0
4,9.256332,267.393719,-1.000000,5.758205,258.911569,76.491274,189.160039,-1.000000,1,1.259486,-1.000000,-1.000000,55.681222,-1.000000,-1.000000,27.768995,73.059709,3.0


In [10]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [11]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [12]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

#### Training 

In [ ]:
#for steps in [int(24e6), ]:
#    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../../final/models/dqn_noisy_4_missing_3_{steps}')
for steps in [int(13e6), int(13.5e6), int(14e6), int(14.5e6), int(15e6), int(15.5e6), int(16e6)]:
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../../final/models/dqn_noisy_6_missing_3_{steps}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.89     |
|    ep_rew_mean      | -0.78    |
|    exploration_rate | 0.793    |
|    success_rate     | 0.15     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 1453     |
|    time_elapsed     | 194      |
|    total_timesteps  | 282641   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 5.78     |
|    n_updates        | 58160    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.04     |
|    ep_rew_mean      | -0.86    |
|    exploration_rate | 0.568    |
|    success_rate     | 0.09     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 1266     |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.46     |
|    ep_rew_mean      | -0.32    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.37     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 1104     |
|    time_elapsed     | 4747     |
|    total_timesteps  | 5243888  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.184    |
|    n_updates        | 1298471  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.7      |
|    ep_rew_mean      | -0.32    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.4      |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 1101     |
|    time_elapsed     | 5083     |
|    total_timesteps  | 5600631  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.09     |
|    ep_rew_mean      | -0.32    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.39     |
| time/               |          |
|    episodes         | 3100000  |
|    fps              | 1070     |
|    time_elapsed     | 9710     |
|    total_timesteps  | 10399540 |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.178    |
|    n_updates        | 2587384  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.05     |
|    ep_rew_mean      | -0.28    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.37     |
| time/               |          |
|    episodes         | 3200000  |
|    fps              | 1071     |
|    time_elapsed     | 10027    |
|    total_timesteps  | 10743174 |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.69     |
|    ep_rew_mean      | -1       |
|    exploration_rate | 0.05     |
|    success_rate     | 0.1      |
| time/               |          |
|    episodes         | 800000   |
|    fps              | 1200     |
|    time_elapsed     | 2041     |
|    total_timesteps  | 2451119  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.463    |
|    n_updates        | 600279   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.1      |
|    ep_rew_mean      | -0.7     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.2      |
| time/               |          |
|    episodes         | 900000   |
|    fps              | 1192     |
|    time_elapsed     | 2311     |
|    total_timesteps  | 2755862  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.51     |
|    ep_rew_mean      | -0.5     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.35     |
| time/               |          |
|    episodes         | 2300000  |
|    fps              | 1102     |
|    time_elapsed     | 6683     |
|    total_timesteps  | 7369252  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0134   |
|    n_updates        | 1829812  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.09     |
|    ep_rew_mean      | -0.46    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.34     |
| time/               |          |
|    episodes         | 2400000  |
|    fps              | 1096     |
|    time_elapsed     | 7033     |
|    total_timesteps  | 7714457  |
| train/              |          |
|    learning_rate  

...

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.27     |
|    ep_rew_mean      | -0.42    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.34     |
| time/               |          |
|    episodes         | 2800000  |
|    fps              | 1123     |
|    time_elapsed     | 8310     |
|    total_timesteps  | 9338305  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0728   |
|    n_updates        | 2322076  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.68     |
|    ep_rew_mean      | -0.5     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.32     |
| time/               |          |
|    episodes         | 2900000  |
|    fps              | 1122     |
|    time_elapsed     | 8639     |
|    total_timesteps  | 9696550  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.15     |
|    ep_rew_mean      | -0.86    |
|    exploration_rate | 0.613    |
|    success_rate     | 0.08     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 1427     |
|    time_elapsed     | 413      |
|    total_timesteps  | 590116   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 10.4     |
|    n_updates        | 135028   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.92     |
|    ep_rew_mean      | -0.82    |
|    exploration_rate | 0.397    |
|    success_rate     | 0.12     |
| time/               |          |
|    episodes         | 300000   |
|    fps              | 1076     |
|    time_elapsed     | 854      |
|    total_timesteps  | 919812   |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.96     |
|    ep_rew_mean      | -0.68    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.17     |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 919      |
|    time_elapsed     | 5728     |
|    total_timesteps  | 5267662  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.127    |
|    n_updates        | 1304415  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.06     |
|    ep_rew_mean      | -0.6     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.2      |
| time/               |          |
|    episodes         | 1800000  |
|    fps              | 917      |
|    time_elapsed     | 6100     |
|    total_timesteps  | 5595945  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.43     |
|    ep_rew_mean      | -0.22    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.42     |
| time/               |          |
|    episodes         | 3200000  |
|    fps              | 930      |
|    time_elapsed     | 11439    |
|    total_timesteps  | 10643822 |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.113    |
|    n_updates        | 2648455  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.71     |
|    ep_rew_mean      | -0.46    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.32     |
| time/               |          |
|    episodes         | 3300000  |
|    fps              | 933      |
|    time_elapsed     | 11799    |
|    total_timesteps  | 11016519 |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.3      |
|    ep_rew_mean      | -1       |
|    exploration_rate | 0.05     |
|    success_rate     | 0.05     |
| time/               |          |
|    episodes         | 500000   |
|    fps              | 1271     |
|    time_elapsed     | 1246     |
|    total_timesteps  | 1585049  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 35.8     |
|    n_updates        | 383762   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.08     |
|    ep_rew_mean      | -1       |
|    exploration_rate | 0.05     |
|    success_rate     | 0.03     |
| time/               |          |
|    episodes         | 600000   |
|    fps              | 1236     |
|    time_elapsed     | 1541     |
|    total_timesteps  | 1905352  |
| train/              |          |
|    learning_rate  

In [ ]:
# class_dict = constants.CLASS_DICT
# df['label'] = df['label'].replace(class_dict)
# X = df.iloc[:, 0:-1]
# y = df.iloc[:, -1]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
# X_train, y_train = np.array(X_train), np.array(y_train)
# X_test, y_test = np.array(X_test), np.array(y_test)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# %%time
# timesteps = int(2e6)
# dqn_model = utils.stable_dqn3(X_train, y_train, timesteps, True, f'../../models/many_features/stable_dqn3_{timesteps}')
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

#### Testing

In [ ]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_6_16000000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

In [ ]:
test_df.y_pred.unique()

In [ ]:
test_df[test_df.y_pred==4]

#### Saving files

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/correlated/test_df3_9000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/correlated/success_df3_9000000.csv', index=False)

#### Confusion matrix and classification report

In [ ]:
test_df = pd.read_csv('../../test_dfs/many_features/0.1/test_df3_6500000.csv')
test_df.head()

In [ ]:
utils.plot_classification_report(test_df['y_actual'], test_df['y_pred'])

In [ ]:
# def plot_confusion_matrix(y_actual, y_pred, save=False, filename=False):
#     from sklearn.metrics import confusion_matrix
#     cm = confusion_matrix(y_actual, y_pred)
#     cm_df = pd.DataFrame(cm, index = [0, 1, 2, 3, 4, 5, 6], columns = [0, 1, 2, 3, 4, 5, 6], dtype='object')
#     #cm_df = pd.DataFrame(cm, index = constants.CLASS_DICT.keys(), columns = constants.CLASS_DICT.keys())
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(cm_df, annot=True)
#     plt.title('Confusion Matrix')
#     plt.ylabel('Actual Anemia')
#     plt.xlabel('Predicted Anemia')
#     plt.tight_layout()
#     if save:
#         plt.savefig(filename)
#     plt.show()
#     plt.close()

In [ ]:
utils.plot_confusion_matrix(test_df['y_actual'], test_df['y_pred'])